In [17]:
!pip install tiktoken

In [3]:
!pip install torch

# Creating a Generative Pretrained Autoencoder


In [36]:
#

# Our Data

In [1]:
import urllib.request
input_file, HTTP_reply = urllib.request.urlretrieve('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt','input.txt')
#print (HTTP_reply)

## Loading the Data
after obtaining the text file, we will load the raw data and use it to create our dataset

In [2]:
with open(input_file, 'r', encoding='utf-8') as f:
    raw_data = f.read()

## Playing With the Data


### trying a sample
let's print the first 10 lines of the dataset

In [3]:
head = ''
first_20_lines = raw_data.split('\n')[:20]
first_20_lines = [l+'\n' for l in first_20_lines]
head = head.join(first_20_lines)
print(head)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.



let's find out what characters appear in the text

In [4]:
chars = sorted(list(set(raw_data)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


# Data processing

## Tokenizer
first we will use a very simple character level encoding-decoding scheme. later on we will examine the differences and benefits of using a more sophisticated tokenizer.

In [5]:
vocab_size = len(chars)
stoi = {c:i for i, c in enumerate(chars)}
itos = {i:c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda t: ''.join([itos[i] for i in t])

lets verify the encoder correctness condition: $$ D(x) = E^{-1}(x) \rightarrow D(E(x)) = x $$

In [2]:
print(decode(encode("ahoy")))

NameError: name 'decode' is not defined

### another tokenizer

In [7]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
assert enc.decode(enc.encode("hello world")) == "hello world"


### encode the data

In [1]:
data = torch.tensor(encode(raw_data))
print (data[:20])

NameError: name 'torch' is not defined

## Separating to Train and Validation Sets

In [ ]:
N = len(data)
BLOCK_SIZE  = 40
BATCH_SIZE = 5
N_TRAIN = int(0.9*N)
train = data[:N_TRAIN]
val = data[N_TRAIN:]


## Dataloader

In [ ]:
def get_batch(dataset,batch_size = BATCH_SIZE):
    n = len(dataset)
    indices = torch.randint(n - BLOCK_SIZE, (batch_size,))
    x = torch.stack([dataset[i : i + BLOCK_SIZE] for i in indices])
    y = torch.stack([dataset[i+1 : i + BLOCK_SIZE +1 ]  for i in indices ])

    return x,y


In [ ]:
batch = get_batch(train)
print(batch)

In [ ]:
print(batch[0].size())

In [ ]:
d = ''
ys =batch[1].tolist()
print(ys)
# for i,s in enumerate(batch[0]):
#     d = decode(s.tolist()+[ys[i]])
#     print(f'{i}. {d}\n')

seems legit.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class BigramNLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, x, targets=None):

        logits = self.token_embedding_table(x)

        B, T, C = logits.size()
        if targets is not None:
            #pytorch cross entropy loss expects a 2d tensor of shape (B*T, C)
            logits = logits.view(B*T,C)

            # same for targets, we flatten it to a 1d tensor of shape (B*T)
            targets = targets.view(B*T)
            #cross entropy loss
            loss = F.cross_entropy(logits,targets)
            return logits, loss
        else:
            return logits, None

    def generate(self, x, N):
        print('hey')
        """"
        x is a 1d tensor of length T
        n is the number of tokens to generate
        returns a 1d tensor of length T+n
        """
        T = x.size(0)
        for i in range(N):
            logits, _ = self.forward(x)
            print("hey: ",logits)
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            x = torch.cat([x, next_token])
            logits, _ = self.forward(x)
            # we only care about the last token
            logits = logits[-1]
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            #
            next_x = torch.multinomial(probs, num_samples=1)
            torch.cat((x, next_x), dim=0)
            print(x)
        return x



In [ ]:
model = BigramNLM(vocab_size)

logits, loss = model(batch[0], batch[1])

# Training

In [ ]:
## using adam optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for step in range(10):

    #load data to train on
    x, y = get_batch(train)

    # zero the gradients
    optimizer.zero_grad(set_to_none=True)

    # forward pass
    logits, loss = model(batch[0], batch[1])

    loss.backward()
    optimizer.step()
    if step % 1000 == 0:
        print(f'step: {step}, loss: {loss.item()}')



In [ ]:
generated = model.generate(batch[0], 100)

In [ ]:
print(generated)